In [3]:
import keras 
from keras.layers import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [22]:
import tensorflow as tf

In [192]:
import os

In [193]:
import struct
import scipy.misc

In [146]:
class MnistHandler():
    def __init__(self,X,y,per=0.9,shuffle=True):
        '''
        this class takes into consideration that the data you give it may be unshuffled
        '''
        self.train_data = X
        self.label = y
        if(shuffle):
            ind = np.arange(self.train_data.shape[0])
            np.random.shuffle(ind)
            self.train_data = self.train_data [ind]
            self.label = self.label [ind]
        row = round(self.train_data.shape[0]*per)
        self.test_data = self.train_data[row:]
        self.label_test = self.label[row:]
        self.train_data = self.train_data[:row]
        self.label = self.label[:row]
        self.crusor = 0
    def next_train_batch(self, batch_size):
        if(self.crusor + batch_size > self.train_data.shape[0]):
            ind = np.arange(self.train_data.shape[0])
            np.random.shuffle(ind)
            self.train_data = self.train_data[ind]
            self.label = self.label[ind]
            self.crusor = 0
        start = self.crusor
        end = start + batch_size
        self.crusor = end
        return (self.train_data[start:end],self.label[start:end])
    
    def get_test_data(self):
        return (self.test_data,self.label_test)
        


In [332]:
def render_fonts_image(x, path, img_per_row, unit_scale=True):
    x = x.reshape((x.shape[0],x.shape[1],x.shape[2]))
    if unit_scale:
        # scale 0-1 matrix back to gray scale bitmaps
        # this can be used to rescale back
        bitmaps = ((x+1)/2.0 * 255.).astype(dtype=np.int16) % 256
    else:
        bitmaps = x
    num_imgs, h, w  = x.shape
    width = img_per_row * w
    height = int(np.ceil(float(num_imgs) / img_per_row)) * h
    canvas = np.zeros(shape=(height, width), dtype=np.int16)
    # make the canvas all white
    canvas.fill(0)
    for idx, bm in enumerate(bitmaps):
        x = h * int(idx / img_per_row)
        y = w * int(idx % img_per_row)
        canvas[x: x + h, y: y + w] = bm
    scipy.misc.toimage(canvas).save(path)
    return path

In [333]:
def read_data():
    '''
    this function returns the shuffled X_train,label_train
    '''
    dfTR = pd.read_csv('train.csv')
    print('data was read')
    dfTR = dfTR.as_matrix().astype(np.float32)
    label = dfTR[:,0]
    dfTR = dfTR[:,1:]
    ind = np.arange(dfTR.shape[0])
    np.random.shuffle(ind)
    dfTR = dfTR[ind]
    # the data is scaled to the out put of tanh which we used as the last activation function in the generator
    dfTR = np.array([i.reshape(28,28,1)/255 *2 -1 for i in dfTR])
    
    label = label[ind]
    return dfTR,label

In [284]:
def check_path(path):
    if not os.path.exists(path):
        os.mkdir(path)

In [334]:
def generator(input, random_dim, is_train, stddevWanted=0.02, reuse=False,show=False):
    '''
    this function generetes fake pictures but tries to make the pictures look like real ones (kinda acts like a forger)
    if you want to use this function more than once make sure that you give reuse = True
    input is the place holder that we want to give to the generator and it's the noise : noise.shape = random_dim
    is_train is for the use of batch norm , so that it can use the right avg and variance in the hidden layers
    stddevWanted is the stddev wanted , may be needed for vanishing and exploading gradiants
    '''
    cs = [128, 64, 32, 16]
    convs = []
    bns = []
    acts = []
    s = 4
    output_dim = 1
    with tf.variable_scope('gan') as scope :
        if reuse :
            scope.reuse_variables()
        '''
        w1 is what connects the noise to the first layer that will later be the foundation of the conv layers
        b1 is the bias term
        '''
        w1 = tf.get_variable('w1', shape=[random_dim,cs[0]*s*s],dtype=tf.float32,initializer=tf.truncated_normal_initializer(stddev=stddevWanted))
        b1 = tf.get_variable('b1',shape=[cs[0]*s*s],dtype=tf.float32,initializer=tf.constant_initializer(0.0))
        
        flat_conv = tf.add(tf.matmul(input,w1),b1,name='flat_conv')
        convs.append(tf.reshape(flat_conv,shape=[-1,s,s,cs[0]],name='conv0'))
        bns.append(tf.layers.batch_normalization(convs[0],training=is_train,name='bn0'))
        acts.append(tf.nn.relu(bns[0],name='act0'))
        if(show):
            print("the 0 shape is :")
            print(acts[0].shape)
        '''
        this part is implimented so that later we can change the code so it can generate any kind of picture :)
        '''
        for i in range(1,4):
            convs.append(tf.layers.conv2d_transpose(acts[-1],cs[i],kernel_size=[5,5],strides=[2,2],padding="SAME",kernel_initializer=tf.truncated_normal_initializer(stddev=stddevWanted),name='conv'+str(i)))
            bns.append(tf.layers.batch_normalization(convs[-1],training=is_train,name='bn'+str(i)))
            acts.append(tf.nn.relu(bns[-1],name='act'+str(i)))
            if(show):
                print("the "+str(i)+" shape is :")
                print(acts[-1].shape)
        conv4 = tf.layers.conv2d_transpose(acts[-1],output_dim,kernel_size=[32,32],strides=[1,1],padding="SAME",kernel_initializer=tf.truncated_normal_initializer(stddev=stddevWanted),name='conv4')
        actLast = tf.nn.tanh(conv4,name='act4')
        if(show):
            print("the last shape is :")
            print(actLast.shape)
        return actLast
        
        

In [335]:
tf.reset_default_graph()
noise = tf.placeholder(shape=[None,100],dtype=tf.float32,name='noise')
a = generator(noise,100,True,reuse=False,show=True)


the 0 shape is :
(?, 4, 4, 128)
the 1 shape is :
(?, 8, 8, 64)
the 2 shape is :
(?, 16, 16, 32)
the 3 shape is :
(?, 32, 32, 16)
the last shape is :
(?, 32, 32, 1)


In [336]:
def lrelu(x, n, leak=0.2):
    '''
    x is the input tensor and n is the name of this layer that you want to give it
    '''
    return tf.maximum(x,leak*x,name=n)

In [337]:
def discriminator (input, is_train,stddevWanted = 0.02, reuse=False,show=False):
    '''
    this function decides if a picture is a real or fake (kinda acts like a police)
    '''
    cs = [16,32,64]
    convs = []
    bns = []
    acts = []
    with tf.variable_scope('dis') as scope :
        if(reuse):
            scope.reuse_variables()
        convs.append(tf.layers.conv2d(input,cs[0],kernel_size=[4,4],strides=[2,2],padding="SAME",kernel_initializer=tf.truncated_normal_initializer(stddev=stddevWanted),name='conv0'))
        bns.append(tf.layers.batch_normalization(convs[-1],training=is_train,name='bn0'))
        acts.append(lrelu(bns[-1],'act0'))
        if(show):
            print("the shape for the 0 layer is :")
            print(acts[-1].shape)
        for i in range(1,len(cs)):
            convs.append(tf.layers.conv2d(acts[-1],cs[i],kernel_size=[4,4],strides=[2,2],padding='SAME',kernel_initializer=tf.truncated_normal_initializer(stddev=stddevWanted),name='conv'+str(i)))
            bns.append(tf.layers.batch_normalization(convs[-1],training=is_train,name='bn'+str(i)))
            acts.append(lrelu(bns[-1],n='act'+str(i)))
            if(show):
                print("the shape for the "+ str(i) +" layer is :")
                print(acts[-1].shape)
        shape = acts[-1].get_shape().as_list()
        dim = shape[1] * shape[2] * shape[3]
        fc1 = tf.reshape(acts[-1],shape=[-1,dim],name = 'fc1')
        w1 = tf.get_variable('w1',shape=[fc1.shape[1],1],dtype=tf.float32,initializer=tf.truncated_normal_initializer(stddev=stddevWanted))
        b1 = tf.get_variable('b1',shape=[1],dtype=tf.float32,initializer=tf.constant_initializer(0.))
        
        out = tf.add(tf.matmul(fc1,w1),b1,name='out')
        
        return out

In [338]:
tf.reset_default_graph()
images = tf.placeholder(shape=[None,28,28,1],dtype=tf.float32,name='image')
a = discriminator(images,True,reuse=False,show=True)


the shape for the 0 layer is :
(?, 14, 14, 16)
the shape for the 1 layer is :
(?, 7, 7, 32)
the shape for the 2 layer is :
(?, 4, 4, 64)


In [347]:
def train(data_path,mode,model_dir,batch_size,rand_dim,num_epochs,lr=0.0001,clip_val=0.01):
    with tf.variable_scope('input'):
        real_image = tf.placeholder(tf.float32,[None,32,32,1],name='real_data')
        noise = tf.placeholder(tf.float32,[None,rand_dim],name='noise')
        is_train = tf.placeholder(tf.bool,name='is_train')
    fake_image = generator(noise,rand_dim,is_train)
    real_result = discriminator(real_image,is_train)
    fake_result = discriminator(fake_image,is_train,reuse=True)
    d_loss = tf.reduce_mean(real_result - fake_result)
    g_loss = tf.reduce_mean(fake_result)
    
    
    
    t_vars = tf.trainable_variables()
    g_vars = [var for var in t_vars if 'gan' in var.name]
    d_vars = [var for var in t_vars if 'dis' in var.name]
    
    trainer_d = tf.train.RMSPropOptimizer(learning_rate=lr).minimize(d_loss,var_list=d_vars)
    trainer_g = tf.train.RMSPropOptimizer(learning_rate=lr).minimize(g_loss, var_list=g_vars)
    
    
    d_clip = [var.assign(tf.clip_by_value(var,-clip_val,clip_val)) for var in d_vars]
    
    
    X,y = read_data()
    
    mnist_data = MnistHandler(X,y)
    
    batch_num = int(mnist_data.train_data.shape[0]/batch_size)
    total_batch = 0 
    sess = tf.Session()
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    
    print('starting to do the magic ...')
    for i in range(epoch_num):
        for j in range(batch_num):
            d_iters = 5
            if (total_batch %500 == 0 or total_batch < 25) :
                d_iters =25
            for k in range(d_iters):
                train_image,_ = mnist_data.next_train_batch(batch_size)
                train_image = np.pad(train_image,((0,0),(2,2),(2,2),(0,0)),'constant',constant_values=(-1,-1))
                train_nois = np.random.uniform(-1.0,1.0,size=[batch_size,rand_dim]).astype(np.float32)
                sess.run(d_clip)
                _,dLoss = sess.run([trainer_d, d_loss],feed_dict={noise:train_nois,real_image:train_image,is_train:True})
                
            train_noise = np.random.uniform(-1.0, +1.0, size=[batch_size,rand_dim])
            _,gLoss = sess.run([trainer_g,g_loss],feed_dict={noise:train_noise,is_train:True})
            print("%d of epochs and %d of batch "%(i,j))
            print("dloss : "+str(dLoss)+" gLoss : "+str(gLoss))

            total_batch += 1
        all_test,_ = mnist_data.get_test_data()
        num_in_test = 100
        ind = np.random.randint(0,all_test.shape[0],num_in_test)
        all_test = all_test[ind]
        all_test = np.pad(all_test,((0,0),(2,2),(2,2),(0,0)),'constant',constant_values=(-1,-1))
        test_noise = np.random.uniform(-1.0, +1.0, size=[num_in_test,rand_dim]).astype(np.float32)
        test_gLoss,test_dLoss,genImages=sess.run([g_loss,d_loss,fake_image],feed_dict={is_train:True,noise:test_noise,real_image:all_test})
        print("dloss : "+str(dLoss)+" gLoss : "+str(gLoss))
        curr_path = os.path.join(model_dir,str(i)+'.jpg')
        render_fonts_image(genImages,curr_path,10)
    saver.save(sess,'model/wgan.cptk')

In [350]:
tf.reset_default_graph()
data_path = 'data'
mode = 'train'
model_dir = 'models'
batch_size = 16
epoch_num = 15
rand_dim = 128
check_path(model_dir)
check_path(data_path)
tf.reset_default_graph()
train(data_path,mode,model_dir,batch_size,rand_dim,epoch_num)

data was read
starting to do the magic ...
0 of epochs and 0 of batch 
dloss : -9.436456e-07 gLoss : 2.4543451e-06
0 of epochs and 1 of batch 
dloss : -1.3487717e-06 gLoss : 2.9796956e-06
0 of epochs and 2 of batch 
dloss : -1.6286007e-06 gLoss : 2.0422851e-06
0 of epochs and 3 of batch 
dloss : -2.0066082e-06 gLoss : 6.4514443e-06
0 of epochs and 4 of batch 
dloss : -2.450178e-06 gLoss : 1.0924932e-05
0 of epochs and 5 of batch 
dloss : -3.046005e-06 gLoss : 1.5927824e-05
0 of epochs and 6 of batch 
dloss : -3.2449461e-06 gLoss : -9.721557e-07
0 of epochs and 7 of batch 
dloss : -5.177255e-06 gLoss : 3.356239e-06
0 of epochs and 8 of batch 
dloss : -1.3183958e-05 gLoss : 4.1933075e-05
0 of epochs and 9 of batch 
dloss : -5.5949906e-05 gLoss : 9.513987e-05
0 of epochs and 10 of batch 
dloss : -0.00029847983 gLoss : 0.00030178792
0 of epochs and 11 of batch 
dloss : -0.0014622072 gLoss : 0.0013657887
0 of epochs and 12 of batch 
dloss : -0.004474932 gLoss : 0.004276643
0 of epochs and 1

0 of epochs and 116 of batch 
dloss : -0.0044690603 gLoss : -0.0015324601
0 of epochs and 117 of batch 
dloss : -0.0059464714 gLoss : -0.0016797634
0 of epochs and 118 of batch 
dloss : -0.004360404 gLoss : -0.0018015393
0 of epochs and 119 of batch 
dloss : -0.0061826482 gLoss : -0.0017552185
0 of epochs and 120 of batch 
dloss : -0.005565835 gLoss : -0.0021423022
0 of epochs and 121 of batch 
dloss : -0.0045440453 gLoss : -0.002133545
0 of epochs and 122 of batch 
dloss : -0.005726359 gLoss : -0.0023354874
0 of epochs and 123 of batch 
dloss : -0.0059560323 gLoss : -0.002805312
0 of epochs and 124 of batch 
dloss : -0.0045102686 gLoss : -0.0026420283
0 of epochs and 125 of batch 
dloss : -0.005148949 gLoss : -0.0022917204
0 of epochs and 126 of batch 
dloss : -0.0048594177 gLoss : -0.0029407605
0 of epochs and 127 of batch 
dloss : -0.005599689 gLoss : -0.0030159354
0 of epochs and 128 of batch 
dloss : -0.004544474 gLoss : -0.002668448
0 of epochs and 129 of batch 
dloss : -0.004285

0 of epochs and 228 of batch 
dloss : -0.00023271496 gLoss : -0.0018949212
0 of epochs and 229 of batch 
dloss : -0.00042847352 gLoss : -0.002113194
0 of epochs and 230 of batch 
dloss : -0.000344368 gLoss : -0.0023314087
0 of epochs and 231 of batch 
dloss : -0.00037724216 gLoss : -0.0025961064
0 of epochs and 232 of batch 
dloss : -0.00021377041 gLoss : -0.0028864206
0 of epochs and 233 of batch 
dloss : -0.00030401105 gLoss : -0.0030214675
0 of epochs and 234 of batch 
dloss : -0.00034641306 gLoss : -0.003160277
0 of epochs and 235 of batch 
dloss : -0.00029014802 gLoss : -0.003247737
0 of epochs and 236 of batch 
dloss : -0.00032780753 gLoss : -0.003427493
0 of epochs and 237 of batch 
dloss : -0.000393646 gLoss : -0.0038284506
0 of epochs and 238 of batch 
dloss : -0.0003628853 gLoss : -0.0038146651
0 of epochs and 239 of batch 
dloss : -0.0002649446 gLoss : -0.003961213
0 of epochs and 240 of batch 
dloss : -0.0004940942 gLoss : -0.004322398
0 of epochs and 241 of batch 
dloss : 

0 of epochs and 340 of batch 
dloss : -0.0005057916 gLoss : -0.0039665517
0 of epochs and 341 of batch 
dloss : -0.000586841 gLoss : -0.0045663454
0 of epochs and 342 of batch 
dloss : -0.00093835336 gLoss : -0.00499923
0 of epochs and 343 of batch 
dloss : -0.0010347192 gLoss : -0.0051391562
0 of epochs and 344 of batch 
dloss : -0.0008474749 gLoss : -0.004538308
0 of epochs and 345 of batch 
dloss : -0.0010120254 gLoss : -0.0041295364
0 of epochs and 346 of batch 
dloss : -0.0010057263 gLoss : -0.0036399951
0 of epochs and 347 of batch 
dloss : -0.0015051889 gLoss : -0.0030939267
0 of epochs and 348 of batch 
dloss : -0.001522101 gLoss : -0.0025014393
0 of epochs and 349 of batch 
dloss : -0.00140302 gLoss : -0.002197918
0 of epochs and 350 of batch 
dloss : -0.0013717181 gLoss : -0.0020412405
0 of epochs and 351 of batch 
dloss : -0.0013646203 gLoss : -0.0022867038
0 of epochs and 352 of batch 
dloss : -0.0016278694 gLoss : -0.0023977547
0 of epochs and 353 of batch 
dloss : -0.0011

0 of epochs and 452 of batch 
dloss : -5.866344e-05 gLoss : -0.0027323302
0 of epochs and 453 of batch 
dloss : -0.00042444543 gLoss : -0.002608975
0 of epochs and 454 of batch 
dloss : -0.0006854845 gLoss : -0.002465083
0 of epochs and 455 of batch 
dloss : -0.0010108592 gLoss : -0.0023170104
0 of epochs and 456 of batch 
dloss : -0.0011344496 gLoss : -0.0022508502
0 of epochs and 457 of batch 
dloss : -0.0014214183 gLoss : -0.0022861557
0 of epochs and 458 of batch 
dloss : -0.0015381079 gLoss : -0.002398565
0 of epochs and 459 of batch 
dloss : -0.0015813713 gLoss : -0.0023529795
0 of epochs and 460 of batch 
dloss : -0.0017917487 gLoss : -0.002466346
0 of epochs and 461 of batch 
dloss : -0.0008776658 gLoss : -0.0028423981
0 of epochs and 462 of batch 
dloss : -0.0002622041 gLoss : -0.002934152
0 of epochs and 463 of batch 
dloss : 0.00010689489 gLoss : -0.0030345372
0 of epochs and 464 of batch 
dloss : 0.0003078878 gLoss : -0.0029656773
0 of epochs and 465 of batch 
dloss : 0.000

...

14 of epochs and 1827 of batch 
dloss : -3.7397025e-05 gLoss : -0.009860951
14 of epochs and 1828 of batch 
dloss : -6.254384e-05 gLoss : -0.009722853
14 of epochs and 1829 of batch 
dloss : -6.8652036e-05 gLoss : -0.010161847
14 of epochs and 1830 of batch 
dloss : 2.2963155e-05 gLoss : -0.009994504
14 of epochs and 1831 of batch 
dloss : -5.1983865e-05 gLoss : -0.010443542
14 of epochs and 1832 of batch 
dloss : -3.423373e-05 gLoss : -0.00964157
14 of epochs and 1833 of batch 
dloss : -6.503507e-05 gLoss : -0.009293884
14 of epochs and 1834 of batch 
dloss : -1.0945369e-06 gLoss : -0.009361346
14 of epochs and 1835 of batch 
dloss : -3.455137e-05 gLoss : -0.009647485
14 of epochs and 1836 of batch 
dloss : -3.1018863e-05 gLoss : -0.010228276
14 of epochs and 1837 of batch 
dloss : -5.3563854e-06 gLoss : -0.010331293
14 of epochs and 1838 of batch 
dloss : -4.3315056e-05 gLoss : -0.009827603
14 of epochs and 1839 of batch 
dloss : -6.6355686e-05 gLoss : -0.009347871
14 of epochs and 1

14 of epochs and 1936 of batch 
dloss : -0.00012604566 gLoss : -0.009213388
14 of epochs and 1937 of batch 
dloss : -0.00010142347 gLoss : -0.008908018
14 of epochs and 1938 of batch 
dloss : -2.5299028e-05 gLoss : -0.009486079
14 of epochs and 1939 of batch 
dloss : -9.8687946e-05 gLoss : -0.010164488
14 of epochs and 1940 of batch 
dloss : -7.566443e-05 gLoss : -0.010391435
14 of epochs and 1941 of batch 
dloss : 0.00016200915 gLoss : -0.010516517
14 of epochs and 1942 of batch 
dloss : -1.8274237e-05 gLoss : -0.010184169
14 of epochs and 1943 of batch 
dloss : -4.3292763e-05 gLoss : -0.009462883
14 of epochs and 1944 of batch 
dloss : -4.3135253e-05 gLoss : -0.009189822
14 of epochs and 1945 of batch 
dloss : -3.4480938e-05 gLoss : -0.009342089
14 of epochs and 1946 of batch 
dloss : -7.027673e-05 gLoss : -0.009539872
14 of epochs and 1947 of batch 
dloss : -6.954302e-06 gLoss : -0.0100906445
14 of epochs and 1948 of batch 
dloss : -5.4566714e-05 gLoss : -0.010046633
14 of epochs an

14 of epochs and 2047 of batch 
dloss : -6.952963e-05 gLoss : -0.00940153
14 of epochs and 2048 of batch 
dloss : -6.63572e-05 gLoss : -0.0097153615
14 of epochs and 2049 of batch 
dloss : -3.075821e-05 gLoss : -0.009516306
14 of epochs and 2050 of batch 
dloss : -3.111112e-05 gLoss : -0.009754489
14 of epochs and 2051 of batch 
dloss : -3.4623838e-05 gLoss : -0.009698334
14 of epochs and 2052 of batch 
dloss : -1.588557e-05 gLoss : -0.009890155
14 of epochs and 2053 of batch 
dloss : -1.9086408e-05 gLoss : -0.009865184
14 of epochs and 2054 of batch 
dloss : -3.849785e-05 gLoss : -0.00939983
14 of epochs and 2055 of batch 
dloss : -7.3175936e-05 gLoss : -0.00895688
14 of epochs and 2056 of batch 
dloss : -1.7976854e-06 gLoss : -0.009772456
14 of epochs and 2057 of batch 
dloss : -5.8077276e-05 gLoss : -0.010123935
14 of epochs and 2058 of batch 
dloss : -3.615243e-05 gLoss : -0.010039207
14 of epochs and 2059 of batch 
dloss : -2.426043e-05 gLoss : -0.009530206
14 of epochs and 2060 o

14 of epochs and 2159 of batch 
dloss : -6.22893e-05 gLoss : -0.009191928
14 of epochs and 2160 of batch 
dloss : -0.00010592671 gLoss : -0.008824129
14 of epochs and 2161 of batch 
dloss : -4.4881774e-05 gLoss : -0.00923925
14 of epochs and 2162 of batch 
dloss : -4.780304e-06 gLoss : -0.009660367
14 of epochs and 2163 of batch 
dloss : -5.071092e-05 gLoss : -0.009866582
14 of epochs and 2164 of batch 
dloss : 7.1219984e-06 gLoss : -0.009276146
14 of epochs and 2165 of batch 
dloss : -2.9191317e-05 gLoss : -0.009485551
14 of epochs and 2166 of batch 
dloss : -3.133819e-05 gLoss : -0.00913078
14 of epochs and 2167 of batch 
dloss : -4.138949e-05 gLoss : -0.009040994
14 of epochs and 2168 of batch 
dloss : -3.405317e-05 gLoss : -0.009159636
14 of epochs and 2169 of batch 
dloss : -4.5235152e-05 gLoss : -0.009471692
14 of epochs and 2170 of batch 
dloss : -2.7349219e-05 gLoss : -0.009715417
14 of epochs and 2171 of batch 
dloss : -5.4512173e-05 gLoss : -0.009492904
14 of epochs and 2172 

14 of epochs and 2270 of batch 
dloss : -0.00011439825 gLoss : -0.009861458
14 of epochs and 2271 of batch 
dloss : -4.6010187e-05 gLoss : -0.009436174
14 of epochs and 2272 of batch 
dloss : -1.673802e-05 gLoss : -0.009057006
14 of epochs and 2273 of batch 
dloss : -5.220156e-05 gLoss : -0.008757645
14 of epochs and 2274 of batch 
dloss : -8.471974e-05 gLoss : -0.008515488
14 of epochs and 2275 of batch 
dloss : -2.71108e-05 gLoss : -0.009045204
14 of epochs and 2276 of batch 
dloss : -5.957065e-05 gLoss : -0.009463496
14 of epochs and 2277 of batch 
dloss : -6.392755e-05 gLoss : -0.009673305
14 of epochs and 2278 of batch 
dloss : 2.3048895e-05 gLoss : -0.00975833
14 of epochs and 2279 of batch 
dloss : 5.338574e-06 gLoss : -0.0090838615
14 of epochs and 2280 of batch 
dloss : -0.00011595496 gLoss : -0.00858174
14 of epochs and 2281 of batch 
dloss : -3.9070146e-05 gLoss : -0.008645389
14 of epochs and 2282 of batch 
dloss : 4.391826e-06 gLoss : -0.009178232
14 of epochs and 2283 of 